In [27]:
import random
from collections import Counter

In [28]:
NUM_PLAYERS = 4
BOXES = 10
WIN_BOXES = 4

In [36]:
class Peg:
    def __init__(self, label: str) -> None:
        self.label = label
        self.position = 0
        self.start = False

    def start(self) -> None:
        self.start = True
    
    @property
    def is_start(self) -> bool:
        return self.start
    
    def move(self, dice: int) -> None:
        self.position += dice


class Mensch:
    def __init__(
            self,
            num_player: int = NUM_PLAYERS,
            boxes: int = BOXES,
            win_boxes: int = WIN_BOXES
            ) -> None:
        self.num_player = num_player
        self.boxes = boxes
        self.win_boxes = win_boxes
        self.players = [Peg(f"Player {i+1}") for i in range(num_player)]

    def roll_dice(self) -> int:
        return random.randint(1, 6)
    
    def turn(self):
        current_index = 0

        while True:
            yield self.players[current_index]
            current_index = (current_index + 1) % self.num_player


    def run(self) -> str:
        while True:

            for i in range(self.num_player):
                player = self.players[i]
                dice = self.roll_dice()
                if not player.is_start:
                    if dice != 6:
                        continue
                    else:
                        player.start()
                        dice = self.roll_dice()



    

In [37]:
class Mensch_Simulate:
    def __init__(self, num_game: int = 1000) -> None:
        self.num_game = num_game

    def roll_dice(self) -> int:
        return random.randint(1, 6)
    
    # def run(self) -> int:
    #     game = Mensch()
    #     winner = game.run()
    #     return winner

    def simulate(self) -> Counter:
        winners = []
        for _ in range(self.num_game):
            winners.append(self.roll_dice())
        return Counter(winners)

        



In [38]:
test = Mensch_Simulate()
test.simulate()

TypeError: unsupported operand type(s) for /: 'dict' and 'int'